<center><img src="image.png" width=500></center>
<p>

You've recently started a new position as a Data Engineer at an energy company. Previously, analysts on other teams had to manually retrieve and clean data every quarter to understand changes in the sales and capability of different energy types. This process normally took days and was something that most analytsts dreaded. Your job is to automate this process by building a data pipeline. You'll write this data pipeline to pull data each month, helping to provide more rapid insights and free up time for your data consumers.

You will achieve this using the `pandas` library and its powerful parsing features. You'll be working with two raw files; `electricity_sales.csv` and `electricity_capability_nested.json`. 
    
Below, you'll find a data dictionary for the `electricity_sales.csv` dataset, which you'll be transforming in just a bit. Good luck!

| Field | Data Type |
| :---- | :-------: |
| period  | `str`        |
| stateid | `str` |
| stateDescription | `str` |
| sectorid | `str` |
| sectorName | `str` |
| price | `float` |
| price-units | `str` |

In [50]:
import pandas as pd
import json

In [51]:
# Extract data from a tabular file_format, with pandas.
def extract_tabular_data(file_path: str):
    try:
        if file_path.endswith('.csv'):
            return pd.read_csv(file_path)
        elif file_path.endswith('.parquet'):
            return pd.read_parquet(file_path)
    except:
        print('Warning: Invalid file extension. Please try with .csv or .parquet!')

In [52]:
# Extract and flatten data from a JSON file.
def extract_json_data(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return pd.json_normalize(data)

In [53]:
# Transform electricity sales to find the total amount of electricity sold in the residential and transportation sectors.
def transform_electricity_sales_data(raw_data: pd.DataFrame):
    raw_data.dropna(subset = 'price',inplace = True)
    raw_data = raw_data[raw_data['sectorName'].isin(['residential', 'transportation'])]
    raw_data['month'] = raw_data['period'].str[-2:]
    raw_data['year'] = raw_data['period'].str[:4]
    return raw_data[['year','month','stateid','price','price-units']]

In [54]:
# Load a DataFrame to a file in either CSV or Parquet format.
def load(dataframe: pd.DataFrame, file_path: str):
    try:
        if file_path.endswith('.csv'):
            return dataframe.to_csv(file_path)
        elif file_path.endswith('.parquet'):
            return dataframe.to_parquet(file_path)
    except:
        print('Warning: {filepath} is not a valid file type. Please try again!_')

In [55]:
# Test the functions.
raw_electricity_capability_df = extract_json_data("electricity_capability_nested.json")
raw_electricity_sales_df = extract_tabular_data("electricity_sales.csv")

cleaned_electricity_sales_df = transform_electricity_sales_data(raw_electricity_sales_df)

load(raw_electricity_capability_df, "loaded__electricity_capability.parquet")
load(cleaned_electricity_sales_df, "loaded__electricity_sales.csv")